In [2]:
#set up working dirs
model_folder = './model_checkpoints'
results_folder = './results/week10/'

In [3]:
#imports
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_fscore_support, confusion_matrix, accuracy_score
from torchvision import transforms 
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import nn, optim

In [4]:
#GET DATA
train_datafolder = '../../images/old_uk_not_uk/train'
#valid_datafolder = '../../images//valid'
#test_datafolder = '../../images/dataset/test'

transformations = transforms.Compose([transforms.Resize((224, 224)), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
full_dataset = ImageFolder(train_datafolder,transform=transformations) 
#valid_dataset = ImageFolder(valid_datafolder,transform=transformations) 

In [5]:
full_dataset.classes

['not_uk', 'uk']

In [6]:
import math
#split training set into training and validation
from torch.utils.data.sampler import SubsetRandomSampler

#get indices and shuffle
tr_0_ind = np.where(np.array(full_dataset.targets) == 0)[0]
tr_1_ind = np.where(np.array(full_dataset.targets) == 1)[0]

tr_1_ind = tr_1_ind.tolist()
tr_1_ind = tr_1_ind*math.ceil(len(tr_0_ind)/len(tr_1_ind))

surplus = len(tr_1_ind)-len(tr_0_ind)
tr_1_ind = np.array(tr_1_ind[surplus:])

len(tr_0_ind), len(tr_1_ind), math.floor(len(tr_0_ind)/len(tr_1_ind))

(978827, 978827, 1)

In [7]:
np.random.shuffle(tr_0_ind)
np.random.shuffle(tr_1_ind)

tr, val = int(0.7*len(tr_0_ind)), int(0.2*len(tr_0_ind))
train_indices = np.concatenate((tr_0_ind[:tr], tr_1_ind[:tr]))
valid_indices = np.concatenate((tr_0_ind[tr:tr+val], tr_1_ind[tr:tr+val]))
test_indices = np.concatenate((tr_0_ind[tr+val:], tr_1_ind[tr+val:]))

'''
val_0_ind = np.where(np.array(valid_dataset.targets) == 0)[0]
val_1_ind = np.where(np.array(valid_dataset.targets) == 1)[0]
np.random.shuffle(val_0_ind)
np.random.shuffle(val_1_ind)
val_min = np.min([len(val_0_ind), len(val_1_ind)])
valid_indices = np.concatenate((val_0_ind[:val_min], val_1_ind[:val_min]))
'''
#shuffle the indices in place
np.random.shuffle(train_indices)
np.random.shuffle(valid_indices)
np.random.shuffle(test_indices)

#create samplers
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [8]:
len(train_indices), len(valid_indices), len(test_indices)

(1370356, 391530, 195768)

In [9]:
#Dataloaders
train_loader = DataLoader(full_dataset, batch_size=128, sampler=train_sampler)
valid_loader = DataLoader(full_dataset, batch_size=128, sampler=valid_sampler)
test_loader = DataLoader(full_dataset, batch_size=128, sampler=test_sampler)

In [10]:
len(train_loader), len(valid_loader), len(test_loader)

(10706, 3059, 1530)

In [11]:
#device settings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device in use: {device}')

Device in use: cuda:0


In [12]:
#Model

class myResNet50(nn.Module):
  def __init__(self, should_hook=False):
    super(myResNet50, self).__init__()

    self.should_hook = should_hook

    pretrainedmodel = resnet50(pretrained=True)
    
    #freezes weights
    if not self.should_hook:
      print("Freezing params...")
      for param in pretrainedmodel.parameters():
        param.requires_grad = False

    self.base = nn.Sequential(*list(pretrainedmodel.children())[:-2])

    self.avgpool = pretrainedmodel.avgpool

    #final layer
    infeatures = pretrainedmodel.fc.in_features
    self.fc = nn.Linear(infeatures, 2)

    # placeholder for the gradients
    self.gradients = None
    
  # method for the gradient extraction
  def get_activations_gradient(self):
    return self.gradients
  
  # method for the activation exctraction
  def get_activations(self, x):
    return self.base(x)

  # hook for the gradients of the activations
  def activations_hook(self, grad):
    self.gradients = grad


  def forward(self, x):
    #x.requires_grad = True
    x = self.base(x)
    
    # register the hook
    if self.should_hook:
      h = x.register_hook(self.activations_hook)
    
    # apply the remaining pooling
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    out = self.fc(x)
    return out


model = myResNet50(should_hook=False)
model.to(device)
#print(model)

model_name = 'uk_notuk_classifier_aug_naive'

Freezing params...


In [13]:
#model settings
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [ ]:
#train network

# number of epochs to train the model
n_epochs = 90

valid_loss_min = np.Inf # track change in validation loss

#result files

train_results = results_folder+model_name+'_tr.csv'
val_results = results_folder+model_name+'_val.csv'

with open(train_results, 'a') as train_result:
    header = ",".join(['loss', 'accuracy', 'tn', 'fp', 'fn', 'tp', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'count_0', 'count_1','auroc','\n'])
    train_result.write(header)

with open(val_results, 'a') as val_result:
    header = ",".join(['loss', 'accuracy', 'tn', 'fp', 'fn', 'tp', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'count_0', 'count_1','auroc','saved','\n'])
    val_result.write(header)

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    # keep track of training and validation outputs
    tr_y_pred, tr_y_target = np.array([]), np.array([])
    val_y_pred, val_y_target = np.array([]), np.array([])

    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

        #save data for metrics
        predictions = torch.exp(output)
      
        _, top_classes = predictions.topk(1,dim=1)

        tr_y_target = np.concatenate((tr_y_target,target.cpu()))
        tr_y_pred = np.concatenate((tr_y_pred,top_classes.squeeze(1).cpu()))
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

        #save data for metrics
        predictions = torch.exp(output)
      
        _, top_classes = predictions.topk(1,dim=1)

        val_y_target = np.concatenate((val_y_target,target.cpu()))
        val_y_pred = np.concatenate((val_y_pred,top_classes.squeeze(1).cpu()))
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)

    # calulate metrics
    with open(train_results, 'a') as train_result:
      a, b = confusion_matrix(tr_y_target, tr_y_pred)
      tn, fp, fn, tp = *a, *b
      precision, recall, f1, support = precision_recall_fscore_support(tr_y_target, tr_y_pred)
      precision_0, precision_1, recall_0, recall_1, f1_0, f1_1, count_0, count_1 = *precision, *recall, *f1, *support
      accuracy = accuracy_score(tr_y_target, tr_y_pred)
      auroc = roc_auc_score(tr_y_target, tr_y_pred)
      line = ",".join([str(train_loss), str(accuracy), str(tn), str(fp), str(fn), str(tp), str(precision_0), str(precision_1), str(recall_0), str(recall_1), str(f1_0), str(f1_1), str(count_0), str(count_1), str(auroc), '\n'])
      train_result.write(line)

    with open(val_results, 'a') as val_result:
      a, b = confusion_matrix(val_y_target, val_y_pred)
      tn, fp, fn, tp = *a, *b
      precision, recall, f1, support = precision_recall_fscore_support(val_y_target, val_y_pred)
      precision_0, precision_1, recall_0, recall_1, f1_0, f1_1, count_0, count_1 = *precision, *recall, *f1, *support
      accuracy = accuracy_score(val_y_target, val_y_pred)
      auroc = roc_auc_score(val_y_target, val_y_pred)
      line = ",".join([str(valid_loss), str(accuracy), str(tn), str(fp), str(fn), str(tp), str(precision_0), str(precision_1), str(recall_0), str(recall_1), str(f1_0), str(f1_1), str(count_0), str(count_1), str(auroc), str(valid_loss <= valid_loss_min),'\n'])
      val_result.write(line)
      

        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving updated model ...'.format(valid_loss_min, valid_loss))
        torch.save(model.state_dict(), model_folder+'/'+model_name+'.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.591957 	Validation Loss: 0.566076
Validation loss decreased (inf --> 0.566076).  Saving updated model ...
Epoch: 2 	Training Loss: 0.573197 	Validation Loss: 0.574022
Epoch: 3 	Training Loss: 0.570199 	Validation Loss: 0.559239
Validation loss decreased (0.566076 --> 0.559239).  Saving updated model ...
Epoch: 4 	Training Loss: 0.569195 	Validation Loss: 0.564711
Epoch: 5 	Training Loss: 0.568998 	Validation Loss: 0.561104
Epoch: 6 	Training Loss: 0.568580 	Validation Loss: 0.571621
Epoch: 7 	Training Loss: 0.568914 	Validation Loss: 0.571957
Epoch: 8 	Training Loss: 0.567589 	Validation Loss: 0.556087
Validation loss decreased (0.559239 --> 0.556087).  Saving updated model ...
Epoch: 9 	Training Loss: 0.568559 	Validation Loss: 0.556180
Epoch: 10 	Training Loss: 0.569062 	Validation Loss: 0.578442


In [15]:
#testing model
model.load_state_dict(torch.load(model_folder+'/'+model_name+'.pt'))

model.eval()
test_accuracy = 0.0

y_pred, y_target = np.array([]), np.array([])

test_results = results_folder+model_name+'_test.csv'

with torch.no_grad():
  for test_images, test_labels in test_loader:
      test_images, test_labels = test_images.to(device), test_labels.to(device)
      out = model(test_images)
      predictions = torch.exp(out)
      
      _, top_classes = predictions.topk(1,dim=1)

      y_target = np.concatenate((y_target,test_labels.cpu()))
      y_pred = np.concatenate((y_pred,top_classes.squeeze(1).cpu()))
      

with open(test_results, 'a') as test_result:
  header = ",".join(['accuracy', 'tn', 'fp', 'fn', 'tp', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'count_0', 'count_1','auroc','\n'])
  a, b = confusion_matrix(y_target, y_pred)
  tn, fp, fn, tp = *a, *b
  precision, recall, f1, support = precision_recall_fscore_support(y_target, y_pred)
  precision_0, precision_1, recall_0, recall_1, f1_0, f1_1, count_0, count_1 = *precision, *recall, *f1, *support
  accuracy = accuracy_score(y_target, y_pred)
  auroc = roc_auc_score(y_target, y_pred)
  line = ",".join([str(accuracy), str(tn), str(fp), str(fn), str(tp), str(precision_0), str(precision_1), str(recall_0), str(recall_1), str(f1_0), str(f1_1), str(count_0), str(count_1), str(auroc), '\n'])
  test_result.write(header)
  test_result.write(line)
  print(f"Results in {test_results}")

print(f'Test Accuracy: {accuracy*100:.6f}%')

Results in ./results/week10/uk_notuk_classifier_aug_naive_test.csv
Test Accuracy: 71.253218%
